### Log

In [1]:
import datetime, threading
from threading import Lock

module_version = "[1.00]"
log_file_path = "./log.txt"
_log_lock = Lock()

def LOGD(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[D]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGW(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[W]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGE(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[E]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

             # YEAR/MM/DD HH:MM:SS.UUUUUU PID   VER  TAG your_msg
LOGD("LOGD") # 2020/05/17 09:01:47.947437 2664 [1.00][D] LOGD
LOGW("LOGW") # 2020/05/17 09:01:47.949437 2664 [1.00][W] LOGW
LOGE("LOGE") # 2020/05/17 09:01:47.950437 2664 [1.00][E] LOGE

2020/05/17 20:59:17.566880 2408 [1.00][D] LOGD
2020/05/17 20:59:17.568880 2408 [1.00][W] LOGW
2020/05/17 20:59:17.569880 2408 [1.00][E] LOGE


### Server

In [2]:
import socket, threading

def server_thread():
    port = 10010
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', port))
    server.listen(5)
    while True:
        try:
            conn, addr = server.accept()
            LOGD('new client connected ', addr)
            # conn is a socket.socket type, socket.socket fd=776, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 12345), raddr=('127.0.0.1', 49451)
            # addr is a tuple, ('127.0.0.1', 49451)
            while True:
                try:
                    data = conn.recv(1024)
                    conn.send(b'Server received ' + data)
                except Exception as err:
                    LOGD("remote client closed")
                    conn.close()
                    break
        except Exception as err:
            LOGE(traceback.format_exc())

threading.Thread(target=server_thread).start()

### Client

In [4]:
import socket

try:
    port = 10010
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('localhost', port))
    
    client.send(b'hello from client 123')
    data = client.recv(1024)
    LOGD(data)
        
    client.close()
except Error as err:
    LOGE(err)


2020/05/17 20:59:22.559165 5164 [1.00][D] new client connected  ('127.0.0.1', 51769)
2020/05/17 20:59:22.564166 2408 [1.00][D] b'Server received hello from client 123'
2020/05/17 20:59:22.567166 5164 [1.00][D] remote client closed
